# This file downloads and creates an intermediary dataset prior to the final version which will be used in our analysis

### Necessary Imports

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader as pdr
import yfinance as yf
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import matplotlib.ticker as mtick
import matplotlib.dates as mpl_dates

import warnings
warnings.filterwarnings("ignore")

### Loading List of Firms by Industry

In [2]:
metal = pd.read_csv("../InputData/Metal.csv")
metal = metal.iloc[:17]
metal = metal[["Company", "Ticker"]]
metal_firm = metal["Ticker"].tolist()

energy = pd.read_csv("../InputData/Energy.csv")
energy = energy[["HOLDINGS", "TICKER"]]
energy_firm = energy["TICKER"].tolist()

semi = pd.read_csv("../InputData/Microchips.csv")
semi = semi.iloc[:38]
semi = semi[["Name", "Ticker"]]
semi_firm = semi["Ticker"].tolist()

trans = pd.read_csv("../InputData/Transport.csv")
trans = trans.iloc[:47]
trans = trans[["Name", "Ticker"]]
trans_firm = trans["Ticker"].tolist()

food = pd.read_csv("../InputData/Food.csv")
food = food[["SecurityName", "Identifier"]]
food_firm = food["Identifier"].tolist()

### Downloading stock prices for selected firms

In [3]:
start_date = datetime(2022, 2, 1)
end_date = datetime(2023, 4, 10)


metal_stocks = yf.download(metal_firm, start=start_date, end=end_date)
energy_stocks = yf.download(energy_firm,start=start_date,end=end_date)
semi_stocks = yf.download(semi_firm,start=start_date,end=end_date)
trans_stocks = yf.download(trans_firm,start=start_date,end=end_date)
food_stocks = yf.download(food_firm,start=start_date,end=end_date)

[*********************100%***********************]  17 of 17 completed
[*********************100%***********************]  112 of 112 completed
[*********************100%***********************]  38 of 38 completed
[*********************100%***********************]  47 of 47 completed
[*********************100%***********************]  30 of 30 completed


### Cleaning Stock Data

In [4]:
metal_prices = metal_stocks.filter(like='Adj Close')               
metal_prices.columns = metal_prices.columns.get_level_values(1)    
metal_prices =  metal_prices.stack().swaplevel().sort_index().reset_index()
metal_prices.columns = ['Firm','Date','Adj Close']

energy_prices = energy_stocks.filter(like='Adj Close')               
energy_prices.columns = energy_prices.columns.get_level_values(1)    
energy_prices =  energy_prices.stack().swaplevel().sort_index().reset_index()
energy_prices.columns = ['Firm','Date','Adj Close']

semi_prices = semi_stocks.filter(like='Adj Close')               
semi_prices.columns = semi_prices.columns.get_level_values(1)    
semi_prices =  semi_prices.stack().swaplevel().sort_index().reset_index()
semi_prices.columns = ['Firm','Date','Adj Close']

trans_prices = trans_stocks.filter(like='Adj Close')               
trans_prices.columns = trans_prices.columns.get_level_values(1)    
trans_prices =  trans_prices.stack().swaplevel().sort_index().reset_index()
trans_prices.columns = ['Firm','Date','Adj Close']

food_prices = food_stocks.filter(like='Adj Close')               
food_prices.columns = food_prices.columns.get_level_values(1)    
food_prices =  food_prices.stack().swaplevel().sort_index().reset_index()
food_prices.columns = ['Firm','Date','Adj Close']

food_prices.head(5)

,Firm,Date,Adj Close
0,ADM,2022-02-01,73.719566
1,ADM,2022-02-02,74.851776
2,ADM,2022-02-03,73.856216
3,ADM,2022-02-04,73.895256
4,ADM,2022-02-07,74.552681


### Downloading and cleaning SP500 Returns (Market Returns)

In [5]:
spy = yf.download('^GSPC', start=start_date, end=end_date)
market_ret = spy.filter(like='Adj Close')               
market_ret = market_ret.stack().swaplevel().sort_index().reset_index()
market_ret.columns = ['Firm','Date','Adj Close']
market_ret['Firm'] = "sp500"
market_ret['Daily Returns'] = market_ret['Adj Close'].pct_change()
market_ret.head(5)

[*********************100%***********************]  1 of 1 completed


,Firm,Date,Adj Close,Daily Returns
0,sp500,2022-02-01,4546.540039,NaN
1,sp500,2022-02-02,4589.379883,0.009423
2,sp500,2022-02-03,4477.439941,-0.024391
3,sp500,2022-02-04,4500.529785,0.005157
4,sp500,2022-02-07,4483.870117,-0.003702


### Loading Data of Major Events in the War
Created manually based on [cnn timeline](https://www.cnn.com/interactive/2023/02/europe/russia-ukraine-war-timeline/index.html)

In [6]:
event_dates = pd.read_csv("../InputData/MajorEvents.csv")
event_dates['Date'] = pd.to_datetime(event_dates['Date'])
event_dates.head(5)

,Date,Event
0,2022-02-24,Invasion Announcement
1,2022-02-25,Ukraine Announces It Will Defend Its Territory
2,2022-03-02,Refugee Crisis in Poland
3,2022-03-06,Irpin Bridge Evacuation
4,2022-03-09,Mariupol Hospital Attack


### Calculation of Daily Returns

In [7]:
metal_prices = metal_prices.sort_values(['Firm', 'Date'])
metal_prices['Daily Returns'] = metal_prices.groupby('Firm')['Adj Close'].pct_change()

energy_prices = energy_prices.sort_values(['Firm', 'Date'])
energy_prices['Daily Returns'] = energy_prices.groupby('Firm')['Adj Close'].pct_change()

semi_prices = semi_prices.sort_values(['Firm', 'Date'])
semi_prices['Daily Returns'] = semi_prices.groupby('Firm')['Adj Close'].pct_change()

trans_prices = trans_prices.sort_values(['Firm', 'Date'])
trans_prices['Daily Returns'] = trans_prices.groupby('Firm')['Adj Close'].pct_change()

food_prices = food_prices.sort_values(['Firm', 'Date'])
food_prices['Daily Returns'] = food_prices.groupby('Firm')['Adj Close'].pct_change()

food_prices.head(5)

,Firm,Date,Adj Close,Daily Returns
0,ADM,2022-02-01,73.719566,NaN
1,ADM,2022-02-02,74.851776,0.015358
2,ADM,2022-02-03,73.856216,-0.013300
3,ADM,2022-02-04,73.895256,0.000529
4,ADM,2022-02-07,74.552681,0.008897


### Calculation of Excess Returns

In [8]:
metal_excess_returns = metal_prices.groupby(['Firm', 'Date'])['Daily Returns'].mean() - market_ret.set_index('Date')['Daily Returns']
metal_excess_returns = metal_excess_returns.reset_index()
metal_excess_returns = metal_excess_returns.rename(columns={'Daily Returns': 'Excess Returns'})
metal_excess_returns["Industry"] = "Metal"

energy_excess_returns = energy_prices.groupby(['Firm', 'Date'])['Daily Returns'].mean() - market_ret.set_index('Date')['Daily Returns']
energy_excess_returns = energy_excess_returns.reset_index()
energy_excess_returns = energy_excess_returns.rename(columns={'Daily Returns': 'Excess Returns'})
energy_excess_returns["Industry"] = "Energy"

semi_excess_returns = semi_prices.groupby(['Firm', 'Date'])['Daily Returns'].mean() - market_ret.set_index('Date')['Daily Returns']
semi_excess_returns = semi_excess_returns.reset_index()
semi_excess_returns = semi_excess_returns.rename(columns={'Daily Returns': 'Excess Returns'})
semi_excess_returns["Industry"] = "Semiconductor"

trans_excess_returns = trans_prices.groupby(['Firm', 'Date'])['Daily Returns'].mean() - market_ret.set_index('Date')['Daily Returns']
trans_excess_returns = trans_excess_returns.reset_index()
trans_excess_returns = trans_excess_returns.rename(columns={'Daily Returns': 'Excess Returns'})
trans_excess_returns["Industry"] = "Transport"

food_excess_returns = food_prices.groupby(['Firm', 'Date'])['Daily Returns'].mean() - market_ret.set_index('Date')['Daily Returns']
food_excess_returns = food_excess_returns.reset_index()
food_excess_returns = food_excess_returns.rename(columns={'Daily Returns': 'Excess Returns'})
food_excess_returns["Industry"] = "Food"

food_excess_returns.head(5)

,Firm,Date,Excess Returns,Industry
0,ADM,2022-02-01,NaN,Food
1,ADM,2022-02-02,0.005936,Food
2,ADM,2022-02-03,0.011091,Food
3,ADM,2022-02-04,-0.004628,Food
4,ADM,2022-02-07,0.012598,Food


### Concatenating all industry datasets and saving the resulting dataset

In [9]:
# Concatenate dataframes vertically
big_df = pd.concat([food_excess_returns, trans_excess_returns, semi_excess_returns, energy_excess_returns, metal_excess_returns])
big_df = big_df.reset_index(drop=True)
big_df.to_csv("../OutputData/inter.csv")
big_df.head(10)

,Firm,Date,Excess Returns,Industry
0,ADM,2022-02-01,NaN,Food
1,ADM,2022-02-02,0.005936,Food
2,ADM,2022-02-03,0.011091,Food
3,ADM,2022-02-04,-0.004628,Food
4,ADM,2022-02-07,0.012598,Food
5,ADM,2022-02-08,0.000943,Food
6,ADM,2022-02-09,-0.006824,Food
7,ADM,2022-02-10,0.007246,Food
8,ADM,2022-02-11,0.019362,Food
9,ADM,2022-02-14,-0.000736,Food
